### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

import pandas as pd

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt
    !pip install -qq autogluon
    !pip install Pillow==9.0.0

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
from autogluon.tabular import TabularDataset, TabularPredictor
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.7 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
system.balance_data = 1
system.scale_data = 1

### ------------------------------------------------------------------------------------------------------------

### Base classifier test

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### AutoGluon

In [ ]:
# Train AutoGluon on train data
X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
if system.scale_data:
    X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
xtd = pd.DataFrame(X_train)
xtd['target'] = y_train
clf = TabularPredictor(label='target').fit(xtd)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------